<a href="https://colab.research.google.com/github/dliuprojects/CIND820_Stroke_Project/blob/main/stroke_risk_factors6_googc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension
!pip install matplotlib
!pip install graphviz

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
from google.colab import files
uploaded = files.upload()

Saving stroke preprocessed.arff to stroke preprocessed (1).arff


In [3]:
import pandas as pd
from scipy.io import arff
import numpy as np

In [4]:
data_file = "stroke preprocessed.arff"
data = arff.loadarff(data_file)

In [5]:
df = pd.DataFrame(data[0])
for col in df.columns:
  if df[col].dtype == 'object':
    # Ensure data isn't read as bytes but rather as strings from file
    df[col] = df[col].str.decode('utf-8')
# Examine data types
print(df.dtypes)

"id"                   float64
"gender"                object
"age"                  float64
"hypertension"          object
"heart_disease"         object
"ever_married"          object
"work_type"             object
"residence_type"        object
"avg_glucose_level"    float64
"bmi"                  float64
"smoking_status"        object
"stroke"                object
dtype: object


In [6]:
# Display first 10 rows
df.head(10)

,"""id""","""gender""","""age""","""hypertension""","""heart_disease""","""ever_married""","""work_type""","""residence_type""","""avg_glucose_level""","""bmi""","""smoking_status""","""stroke"""
0,9046.0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676.0,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,5000.0,never smoked,1
2,31112.0,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182.0,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665.0,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669.0,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882.0,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434.0,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419.0,Female,59.0,0,0,Yes,Private,Rural,76.15,5000.0,Unknown,1
9,60491.0,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


In [7]:
# Examine meta info about data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   "id"                 5110 non-null   float64
 1   "gender"             5110 non-null   object 
 2   "age"                5110 non-null   float64
 3   "hypertension"       5110 non-null   object 
 4   "heart_disease"      5110 non-null   object 
 5   "ever_married"       5110 non-null   object 
 6   "work_type"          5110 non-null   object 
 7   "residence_type"     5110 non-null   object 
 8   "avg_glucose_level"  5110 non-null   float64
 9   "bmi"                5110 non-null   float64
 10  "smoking_status"     5110 non-null   object 
 11  "stroke"             5110 non-null   object 
dtypes: float64(4), object(8)
memory usage: 479.2+ KB


In [8]:
# The original 201 null values were all from bmi column, and they have been replaced by placeholder value 5000
# Convert the 5000 values back into null values
df = df.replace(5000.0, np.nan)

In [9]:
# Check head of dataset again
df.head(10)

,"""id""","""gender""","""age""","""hypertension""","""heart_disease""","""ever_married""","""work_type""","""residence_type""","""avg_glucose_level""","""bmi""","""smoking_status""","""stroke"""
0,9046.0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676.0,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112.0,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182.0,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665.0,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669.0,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882.0,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434.0,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419.0,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,60491.0,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


In [10]:
# Check structure of data types to ensure bmi remains float
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   "id"                 5110 non-null   float64
 1   "gender"             5110 non-null   object 
 2   "age"                5110 non-null   float64
 3   "hypertension"       5110 non-null   object 
 4   "heart_disease"      5110 non-null   object 
 5   "ever_married"       5110 non-null   object 
 6   "work_type"          5110 non-null   object 
 7   "residence_type"     5110 non-null   object 
 8   "avg_glucose_level"  5110 non-null   float64
 9   "bmi"                4909 non-null   float64
 10  "smoking_status"     5110 non-null   object 
 11  "stroke"             5110 non-null   object 
dtypes: float64(4), object(8)
memory usage: 479.2+ KB


In [11]:
# Remove records with NAs from dataset
df_noNA = df
df_noNA = df_noNA.dropna()
df_noNA.head(10)

,"""id""","""gender""","""age""","""hypertension""","""heart_disease""","""ever_married""","""work_type""","""residence_type""","""avg_glucose_level""","""bmi""","""smoking_status""","""stroke"""
0,9046.0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,31112.0,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182.0,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665.0,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669.0,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882.0,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434.0,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
9,60491.0,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1
10,12109.0,Female,81.0,1,0,Yes,Private,Rural,80.43,29.7,never smoked,1
11,12095.0,Female,61.0,0,1,Yes,Govt_job,Rural,120.46,36.8,smokes,1


In [12]:
# Modify 'stroke' column so that 0 is coded as "No Stroke" and 1 is coded as "Stroke"
df_noNA.loc[df_noNA['"stroke"'] == '0', '"stroke"'] = "No Stroke"
df_noNA.loc[df_noNA['"stroke"'] == '1', '"stroke"'] = "Stroke"
df_noNA.head(10)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,"""id""","""gender""","""age""","""hypertension""","""heart_disease""","""ever_married""","""work_type""","""residence_type""","""avg_glucose_level""","""bmi""","""smoking_status""","""stroke"""
0,9046.0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,Stroke
2,31112.0,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,Stroke
3,60182.0,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,Stroke
4,1665.0,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,Stroke
5,56669.0,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,Stroke
6,53882.0,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,Stroke
7,10434.0,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,Stroke
9,60491.0,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,Stroke
10,12109.0,Female,81.0,1,0,Yes,Private,Rural,80.43,29.7,never smoked,Stroke
11,12095.0,Female,61.0,0,1,Yes,Govt_job,Rural,120.46,36.8,smokes,Stroke


In [13]:
 # See if there are any extreme values in numeric data
 df_noNA.describe()

,"""id""","""age""","""avg_glucose_level""","""bmi"""
count,4909.000000,4909.000000,4909.000000,4909.000000
mean,37064.313506,42.865374,105.305150,28.893237
std,20995.098457,22.555115,44.424341,7.854067
min,77.000000,0.080000,55.120000,10.300000
25%,18605.000000,25.000000,77.070000,23.500000
50%,37608.000000,44.000000,91.680000,28.100000
75%,55220.000000,60.000000,113.570000,33.100000
max,72940.000000,82.000000,271.740000,97.600000


In [14]:
# Create list of categorical columns
cat_cols = ['"gender"', '"hypertension"', '"heart_disease"', '"ever_married"', '"work_type"', '"residence_type"', '"smoking_status"']

In [15]:
# Create copy of a data frame in memory w/ a different name
df_dummy = df_noNA.copy()
# Convert only categorical feature into dummy/one-hot features
df_dummy = pd.get_dummies(df_noNA, columns = cat_cols, prefix = cat_cols)
# Print dataset
df_dummy

,"""id""","""age""","""avg_glucose_level""","""bmi""","""stroke""","""gender""_Female","""gender""_Male","""gender""_Other","""hypertension""_0","""hypertension""_1",...,"""work_type""_Never_worked","""work_type""_Private","""work_type""_Self-employed","""work_type""_children","""residence_type""_Rural","""residence_type""_Urban","""smoking_status""_Unknown","""smoking_status""_formerly smoked","""smoking_status""_never smoked","""smoking_status""_smokes"
0,9046.0,67.0,228.69,36.6,Stroke,0,1,0,1,0,...,0,1,0,0,0,1,0,1,0,0
2,31112.0,80.0,105.92,32.5,Stroke,0,1,0,1,0,...,0,1,0,0,1,0,0,0,1,0
3,60182.0,49.0,171.23,34.4,Stroke,1,0,0,1,0,...,0,1,0,0,0,1,0,0,0,1
4,1665.0,79.0,174.12,24.0,Stroke,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
5,56669.0,81.0,186.21,29.0,Stroke,0,1,0,1,0,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,14180.0,13.0,103.08,18.6,No Stroke,1,0,0,1,0,...,0,0,0,1,1,0,1,0,0,0
5106,44873.0,81.0,125.20,40.0,No Stroke,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
5107,19723.0,35.0,82.99,30.6,No Stroke,1,0,0,1,0,...,0,0,1,0,1,0,0,0,1,0
5108,37544.0,51.0,166.29,25.6,No Stroke,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0


In [16]:
# Create train test set split
from sklearn.model_selection import train_test_split

In [17]:
# Set class name as "stroke", all else will be used as features
class_col_name = '"stroke"'
dummy_feature_names = df_dummy.columns[df_dummy.columns != class_col_name]
# 70% training, 30% test set split
x_train, x_test, y_train, y_test = train_test_split(df_dummy.loc[:, dummy_feature_names], df_dummy[class_col_name], test_size=0.3, random_state=1)

In [18]:
# Naive Bayes modeling
from sklearn.naive_bayes import MultinomialNB

In [19]:
# Create Multinomial NB Classifier
nb = MultinomialNB()

In [20]:
# Train model using training sets
nb.fit(x_train, y_train)

MultinomialNB()

In [21]:
# Predict response for test dataset
y_pred = nb.predict(x_test)

In [22]:
# Print Naive Bayes output
print("Number of features used: ", nb.n_features_)
print("Classes: ", nb.classes_)
print("Number of records for classes: ", nb.class_count_)
print("Log prior probability for classes: ", nb.class_log_prior_)
print("Log conditional probability for each feature given a class: ", nb.feature_log_prob_)

Number of features used:  24
Classes:  ['No Stroke' 'Stroke']
Number of records for classes:  [3291.  145.]
Log prior probability for classes:  [-0.04311653 -3.16532954]
Log conditional probability for each feature given a class:  [[-4.86773845e-03 -6.79502904e+00 -5.88583704e+00 -7.16791435e+00
  -1.10563808e+01 -1.14237629e+01 -1.79362509e+01 -1.06200350e+01
  -1.29804239e+01 -1.05742404e+01 -1.36665535e+01 -1.15561284e+01
  -1.09735341e+01 -1.25702749e+01 -1.57390263e+01 -1.10803154e+01
  -1.24452492e+01 -1.24852125e+01 -1.12629529e+01 -1.11845648e+01
  -1.17028211e+01 -1.23527546e+01 -1.15025073e+01 -1.24167920e+01]
 [-6.14395678e-03 -6.36670554e+00 -5.66931236e+00 -7.16182860e+00
  -1.11140899e+01 -1.14296068e+01 -1.55567412e+01 -1.09027808e+01
  -1.18190715e+01 -1.07776177e+01 -1.22245366e+01 -1.28486910e+01
  -1.06739392e+01 -1.24656987e+01 -1.55567412e+01 -1.10458817e+01
  -1.20013931e+01 -1.55567412e+01 -1.13670864e+01 -1.11622920e+01
  -1.25610089e+01 -1.17955410e+01 -1.15494

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [23]:
# Confusion matrix and Evaluation metrics
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(y_test, y_pred)
print("Confusion Matrix")
print(cf)
tn, fp, fn, tp = cf.ravel()
print("TP: ", tp, ", FP: ", fp, ", TN: ", tn, ", FN: ", fn)

Confusion Matrix
[[872 537]
 [ 21  43]]
TP:  43 , FP:  537 , TN:  872 , FN:  21


In [24]:
# Classifier report
from sklearn.metrics import classification_report
from sklearn import metrics

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   No Stroke       0.98      0.62      0.76      1409
      Stroke       0.07      0.67      0.13        64

    accuracy                           0.62      1473
   macro avg       0.53      0.65      0.45      1473
weighted avg       0.94      0.62      0.73      1473



In [25]:
# Decision tree on dummy encoded data
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth = 5) # 5 levels set
clf = clf.fit(x_train, y_train)

import graphviz
# Obtain unique class values to show on tree
class_values = df_dummy[class_col_name]. unique()
print("class names: ", class_values)
dot_data = tree.export_graphviz(clf, out_file = None, feature_names = dummy_feature_names, class_names = class_values, filled = True)

class names:  ['Stroke' 'No Stroke']


In [26]:
# Draw graph
graph = graphviz.Source(dot_data, format = "png")
# Graph won't display

In [27]:
# Perform prediction on test set
y_pred = clf.predict(x_test)

In [28]:
# Get decision tree report
from sklearn.metrics import classification_report
from sklearn import metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   No Stroke       0.96      0.99      0.97      1409
      Stroke       0.12      0.03      0.05        64

    accuracy                           0.95      1473
   macro avg       0.54      0.51      0.51      1473
weighted avg       0.92      0.95      0.93      1473

